# Self-Improving AI Tutor with Multi-Agent System and Qdrant

This notebook implements a complete adaptive tutoring system that:
- Generates lesson plans from PDF content
- Uses multiple AI agents that learn from experience
- Adapts teaching style based on student emotions
- Stores all learning experiences in Qdrant vector database
- Includes quiz systems and progress tracking

## Step 1: Install Dependencies

In [ ]:
# Install required packages
! pip install qdrant-client groq pymupdf text2emotion swarm python-dotenv Pillow numpy

In [ ]:
# Install swarm from GitHub
! pip install git+https://github.com/openai/swarm.git

## Step 2: Import Libraries

In [ ]:
import fitz  # PyMuPDF
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance, VectorParams, PointStruct, Filter, 
    FieldCondition, MatchValue, PayloadSchemaType
)
import uuid
import json
import time
import base64
import io
import hashlib
from datetime import datetime
from typing import List, Dict, Optional, Tuple
import numpy as np
from collections import defaultdict
import text2emotion as te
from PIL import Image
from swarm import Swarm, Agent

## Step 3: Configuration and Initialize Clients

In [ ]:
# API Keys
GROQ_API_KEY = "YOUR_GROQ_API_KEY"
QDRANT_URL = "YOUR_QDRANT_URL"
QDRANT_API_KEY = "YOUR_QDRANT_API_KEY"

# Initialize clients
groq_client = Groq(api_key=GROQ_API_KEY)
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
swarm_client = Swarm(client=groq_client)

# Configuration
GROQ_MODEL = "llama-3.3-70b-versatile"
EMBEDDING_DIM = 384
COLLECTION_PDF_CONTENT = "pdf_content_sequential"
COLLECTION_TEACHING_STYLES = "teaching_styles"
COLLECTION_STUDENT_MEMORY = "student_memory"
COLLECTION_AGENT_LEARNING = "agent_learning"
COLLECTION_PDF_IMAGES = "pdf_images"

print(f"Configuration complete")
print(f"Embedding dimension: {EMBEDDING_DIM}")
print(f"Collections: {5}")

## Step 4: Utility Functions

In [ ]:
def simple_embed(text: str, dim: int = 384) -> List[float]:
    """
    Create embeddings using hash-based approach.
    Simple and fast, no PyTorch dependencies needed.
    """
    words = text.lower().split()
    vector = [0.0] * dim
    
    for i, word in enumerate(words[:100]):
        hash_val = hash(word)
        for j in range(3):
            idx = (hash_val + j * 13) % dim
            vector[idx] += 1.0 / (i + 1)
    
    # Normalize
    norm = np.sqrt(sum(v*v for v in vector))
    if norm > 0:
        vector = [v/norm for v in vector]
    
    return vector

print("Embedding function ready")

## Step 5: Setup Qdrant Collections

In [ ]:
def setup_qdrant_collections():
    """
    Create all 5 Qdrant collections with appropriate configurations
    """
    collections = [
        (COLLECTION_PDF_CONTENT, "Sequential PDF content for teaching"),
        (COLLECTION_TEACHING_STYLES, "Teaching styles mapped to emotions"),
        (COLLECTION_STUDENT_MEMORY, "Student progress and preferences"),
        (COLLECTION_AGENT_LEARNING, "Agent learning experiences and outcomes"),
        (COLLECTION_PDF_IMAGES, "PDF images with descriptions")
    ]
    
    for collection_name, description in collections:
        try:
            if qdrant_client.collection_exists(collection_name):
                qdrant_client.delete_collection(collection_name)
                print(f"Deleted existing: {collection_name}")
            
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=EMBEDDING_DIM, distance=Distance.COSINE)
            )
            print(f"Created: {collection_name}")
            
            # Create indexes
            if collection_name == COLLECTION_PDF_CONTENT:
                qdrant_client.create_payload_index(
                    collection_name=collection_name,
                    field_name="sequence_id",
                    field_schema=PayloadSchemaType.INTEGER
                )
                qdrant_client.create_payload_index(
                    collection_name=collection_name,
                    field_name="difficulty",
                    field_schema=PayloadSchemaType.KEYWORD
                )
            elif collection_name == COLLECTION_AGENT_LEARNING:
                qdrant_client.create_payload_index(
                    collection_name=collection_name,
                    field_name="agent_name",
                    field_schema=PayloadSchemaType.KEYWORD
                )
            elif collection_name == COLLECTION_PDF_IMAGES:
                qdrant_client.create_payload_index(
                    collection_name=collection_name,
                    field_name="page",
                    field_schema=PayloadSchemaType.INTEGER
                )
        except Exception as e:
            print(f"Error with {collection_name}: {e}")
    
    print("\nAll collections ready")

setup_qdrant_collections()

## Step 6: PDF Processing

In [ ]:
class EnhancedPDFProcessor:
    """
    Process PDF files to extract text chunks and images
    """
    
    def __init__(self):
        pass
    
    def process_pdf(self, pdf_path: str) -> Tuple[List[Dict], List[Dict]]:
        """
        Extract content and images from PDF with rich metadata
        """
        doc = fitz.open(pdf_path)
        text_chunks = []
        images = []
        chunk_id = 0
        
        print(f"Processing PDF: {pdf_path}")
        print(f"Total pages: {len(doc)}")
        
        for page_num, page in enumerate(doc):
            text = page.get_text()
            
            # Extract images
            image_list = page.get_images(full=True)
            for img_index, img in enumerate(image_list):
                try:
                    xref = img[0]
                    base_image = doc.extract_image(xref)
                    image_bytes = base_image["image"]
                    image_ext = base_image["ext"]
                    
                    image_b64 = base64.b64encode(image_bytes).decode()
                    description = f"Image {len(images)+1} from page {page_num+1}"
                    vector = simple_embed(description)
                    
                    images.append({
                        "id": str(uuid.uuid4()),
                        "vector": vector,
                        "payload": {
                            "description": description,
                            "page": page_num + 1,
                            "image_base64": image_b64,
                            "image_format": image_ext,
                            "image_index": len(images)
                        }
                    })
                except Exception as e:
                    print(f"Could not extract image {img_index} from page {page_num}: {e}")
            
            # Split text into chunks
            sentences = text.split('. ')
            
            for i in range(0, len(sentences), 3):
                chunk_text = '. '.join(sentences[i:i+3]).strip()
                
                if len(chunk_text) < 20:
                    continue
                
                vector = simple_embed(chunk_text)
                word_count = len(chunk_text.split())
                avg_word_length = np.mean([len(w) for w in chunk_text.split()]) if word_count > 0 else 0
                difficulty = "beginner" if avg_word_length < 6 else "intermediate" if avg_word_length < 8 else "advanced"
                
                text_chunks.append({
                    "id": str(uuid.uuid4()),
                    "vector": vector,
                    "payload": {
                        "text": chunk_text,
                        "sequence_id": chunk_id,
                        "page": page_num + 1,
                        "word_count": word_count,
                        "difficulty": difficulty,
                        "timestamp": datetime.now().isoformat()
                    }
                })
                
                chunk_id += 1
        
        doc.close()
        print(f"Extracted {len(text_chunks)} content chunks")
        print(f"Extracted {len(images)} images")
        return text_chunks, images
    
    def ingest_to_qdrant(self, text_chunks: List[Dict], images: List[Dict]):
        """Upload chunks and images to Qdrant"""
        if text_chunks:
            text_points = [
                PointStruct(id=chunk["id"], vector=chunk["vector"], payload=chunk["payload"])
                for chunk in text_chunks
            ]
            qdrant_client.upsert(collection_name=COLLECTION_PDF_CONTENT, points=text_points)
            print(f"Ingested {len(text_points)} text chunks to Qdrant")
        
        if images:
            image_points = [
                PointStruct(id=img["id"], vector=img["vector"], payload=img["payload"])
                for img in images
            ]
            qdrant_client.upsert(collection_name=COLLECTION_PDF_IMAGES, points=image_points)
            print(f"Ingested {len(image_points)} images to Qdrant")

pdf_processor = EnhancedPDFProcessor()
print("PDF Processor ready")

## Step 7: Initialize Teaching Styles

In [ ]:
def initialize_teaching_styles():
    """
    Pre-load teaching styles for different emotions
    """
    styles = [
        {
            "name": "Empathetic and Encouraging",
            "emotion": "sad",
            "description": "Gentle, supportive tone with positive reinforcement",
            "characteristics": "Use encouraging words, acknowledge feelings, break down complex ideas"
        },
        {
            "name": "Enthusiastic and Motivational",
            "emotion": "happy",
            "description": "Energetic, engaging tone that builds on positive momentum",
            "characteristics": "Use exciting examples, challenge with interesting problems"
        },
        {
            "name": "Patient and Clarifying",
            "emotion": "angry",
            "description": "Calm, clear explanations with extra patience",
            "characteristics": "Simplify concepts, use multiple analogies"
        },
        {
            "name": "Clear and Structured",
            "emotion": "fear",
            "description": "Reassuring, well-organized explanations",
            "characteristics": "Step-by-step approach, predictable structure"
        },
        {
            "name": "Balanced and Informative",
            "emotion": "neutral",
            "description": "Standard teaching approach with good examples",
            "characteristics": "Clear explanations, relevant examples"
        }
    ]
    
    points = []
    for style in styles:
        text_for_embedding = f"{style['description']} {style['characteristics']}"
        vector = simple_embed(text_for_embedding)
        points.append(PointStruct(id=str(uuid.uuid4()), vector=vector, payload=style))
    
    qdrant_client.upsert(collection_name=COLLECTION_TEACHING_STYLES, points=points)
    print(f"Initialized {len(styles)} teaching styles")

initialize_teaching_styles()

## Step 8: Core Helper Functions

These functions must be defined before the agents, as agents will call them.

In [ ]:
def detect_emotion(text: str) -> Dict:
    """
    Detect emotion from student's text using text2emotion library
    Returns dominant emotion and all scores
    """
    try:
        emotions = te.get_emotion(text)
        dominant_emotion = max(emotions.items(), key=lambda x: x[1])[0]
        return {
            "dominant": dominant_emotion,
            "scores": emotions
        }
    except:
        return {
            "dominant": "neutral",
            "scores": {}
        }

# Shared context for agents
agent_context = {
    "current_student": None,
    "current_topic": None,
    "conversation_history": [],
    "session_metrics": defaultdict(list)
}

def query_pdf_content(sequence_id: Optional[int] = None, 
                     topic_search: Optional[str] = None,
                     difficulty: Optional[str] = None) -> Dict:
    """
    Query PDF content from Qdrant by sequence ID or semantic search
    """
    if topic_search:
        query_vector = simple_embed(topic_search)
        filter_conditions = []
        
        if difficulty:
            filter_conditions.append(
                FieldCondition(key="difficulty", match=MatchValue(value=difficulty))
            )
        
        results = qdrant_client.query_points(
            collection_name=COLLECTION_PDF_CONTENT,
            query=query_vector,
            query_filter=Filter(must=filter_conditions) if filter_conditions else None,
            limit=1
        )
        
        if results.points:
            return results.points[0].payload
    
    elif sequence_id is not None:
        results = qdrant_client.scroll(
            collection_name=COLLECTION_PDF_CONTENT,
            scroll_filter=Filter(
                must=[FieldCondition(key="sequence_id", match=MatchValue(value=sequence_id))]
            ),
            limit=1
        )
        
        if results[0]:
            return results[0][0].payload
    
    return {"text": "No content found", "sequence_id": -1}

def query_teaching_style(emotion: str) -> Dict:
    """
    Get appropriate teaching style for given emotion from Qdrant
    """
    emotion_query = f"Teaching style for {emotion} emotion"
    query_vector = simple_embed(emotion_query)
    
    results = qdrant_client.query_points(
        collection_name=COLLECTION_TEACHING_STYLES,
        query=query_vector,
        limit=1
    )
    
    if results.points:
        return results.points[0].payload
    
    return {
        "name": "Balanced",
        "description": "Standard teaching approach",
        "characteristics": "Clear explanations with examples"
    }

def query_related_images(page_num: int) -> List[Dict]:
    """
    Query images from specific page for whiteboard display
    """
    results = qdrant_client.scroll(
        collection_name=COLLECTION_PDF_IMAGES,
        scroll_filter=Filter(
            must=[FieldCondition(key="page", match=MatchValue(value=page_num))]
        ),
        limit=5
    )
    
    images = []
    if results[0]:
        for point in results[0]:
            images.append({
                "description": point.payload.get("description"),
                "image_base64": point.payload.get("image_base64"),
                "image_format": point.payload.get("image_format")
            })
    
    return images

print("Helper functions defined successfully")

## Step 9: Student Memory System

Tracks individual student progress and preferences in Qdrant.

In [ ]:
class StudentMemorySystem:
    """
    Track student progress, preferences, and learning history
    """
    
    def __init__(self):
        pass
    
    def _get_uuid_from_string(self, string_id: str) -> str:
        """Generate consistent UUID from string ID"""
        return str(uuid.uuid5(uuid.NAMESPACE_DNS, string_id))
    
    def initialize_student(self, student_id: str):
        """Create initial student profile in Qdrant"""
        vector = simple_embed(f"Student profile: {student_id}")
        point_id = self._get_uuid_from_string(student_id)
        
        point = PointStruct(
            id=point_id,
            vector=vector,
            payload={
                "student_id": student_id,
                "topics_covered": [],
                "confusion_points": [],
                "mastered_topics": [],
                "preferred_styles": [],
                "total_interactions": 0,
                "start_time": datetime.now().isoformat()
            }
        )
        
        qdrant_client.upsert(
            collection_name=COLLECTION_STUDENT_MEMORY,
            points=[point]
        )
        print(f"Student initialized: {student_id}")
    
    def update_student_progress(self, student_id: str, topic: str, 
                               understood: bool, emotion: str, style_used: str):
        """Update student's learning progress"""
        point_id = self._get_uuid_from_string(student_id)
        
        result = qdrant_client.retrieve(
            collection_name=COLLECTION_STUDENT_MEMORY,
            ids=[point_id]
        )
        
        if result:
            current_payload = result[0].payload
            
            if topic not in current_payload["topics_covered"]:
                current_payload["topics_covered"].append(topic)
            
            if understood:
                if topic not in current_payload["mastered_topics"]:
                    current_payload["mastered_topics"].append(topic)
            else:
                if topic not in current_payload["confusion_points"]:
                    current_payload["confusion_points"].append(topic)
            
            if style_used not in current_payload["preferred_styles"]:
                current_payload["preferred_styles"].append(style_used)
            
            current_payload["total_interactions"] += 1
            current_payload["last_emotion"] = emotion
            current_payload["last_update"] = datetime.now().isoformat()
            
            progress_text = f"Student {student_id}: {len(current_payload['mastered_topics'])} mastered"
            vector = simple_embed(progress_text)
            
            qdrant_client.upsert(
                collection_name=COLLECTION_STUDENT_MEMORY,
                points=[PointStruct(
                    id=point_id,
                    vector=vector,
                    payload=current_payload
                )]
            )
    
    def get_student_profile(self, student_id: str) -> Dict:
        """Retrieve complete student profile"""
        point_id = self._get_uuid_from_string(student_id)
        result = qdrant_client.retrieve(
            collection_name=COLLECTION_STUDENT_MEMORY,
            ids=[point_id]
        )
        return result[0].payload if result else None
    
    def recommend_next_topic(self, student_id: str) -> Dict:
        """Recommend next topic based on progress"""
        profile = self.get_student_profile(student_id)
        
        if not profile:
            return {
                "recommendation": "Start from beginning",
                "reason": "New student",
                "type": "new",
                "sequence_id": 0
            }
        
        topics_covered = len(profile.get("topics_covered", []))
        confusion_points = profile.get("confusion_points", [])
        
        if confusion_points:
            return {
                "recommendation": confusion_points[-1],
                "reason": "Review confused topic",
                "type": "review"
            }
        
        return {
            "recommendation": f"Topic {topics_covered + 1}",
            "reason": "Continue progression",
            "type": "next",
            "sequence_id": topics_covered
        }

memory_system = StudentMemorySystem()
print("Student Memory System initialized")

## Step 10: Agent Learning System

This is the KEY INNOVATION - agents store experiences and learn from past successes.

In [ ]:
class AgentLearningSystem:
    """
    Stores and retrieves agent experiences for continuous improvement.
    This enables true agent learning, not just retrieval.
    """
    
    def __init__(self):
        pass
    
    def store_experience(self, agent_name: str, situation: str, action: str, 
                        outcome_score: float, metadata: Dict):
        """
        Store teaching experience in Qdrant for future learning
        
        Args:
            agent_name: Which agent took the action
            situation: Description of the context
            action: What action was taken
            outcome_score: Success metric (0-1)
            metadata: Additional context
        """
        experience_text = f"{situation} | Action: {action}"
        vector = simple_embed(experience_text)
        
        payload = {
            "agent_name": agent_name,
            "situation": situation,
            "action": action,
            "outcome_score": outcome_score,
            "timestamp": datetime.now().isoformat(),
            **metadata
        }
        
        point = PointStruct(
            id=str(uuid.uuid4()),
            vector=vector,
            payload=payload
        )
        
        qdrant_client.upsert(
            collection_name=COLLECTION_AGENT_LEARNING,
            points=[point]
        )
    
    def retrieve_successful_strategies(self, agent_name: str, situation: str, 
                                      top_k: int = 3, min_score: float = 0.6):
        """
        Retrieve past successful strategies for similar situations
        This is how agents LEARN and EVOLVE
        """
        query_vector = simple_embed(situation)
        
        results = qdrant_client.query_points(
            collection_name=COLLECTION_AGENT_LEARNING,
            query=query_vector,
            query_filter=Filter(
                must=[
                    FieldCondition(
                        key="agent_name",
                        match=MatchValue(value=agent_name)
                    )
                ]
            ),
            limit=top_k
        )
        
        successful_strategies = [
            {
                "action": hit.payload["action"],
                "outcome_score": hit.payload["outcome_score"],
                "similarity": hit.score,
                "metadata": {k: v for k, v in hit.payload.items() 
                           if k not in ["action", "outcome_score", "agent_name", "situation"]}
            }
            for hit in results.points
            if hit.payload.get("outcome_score", 0) >= min_score
        ]
        
        return successful_strategies
    
    def get_agent_performance_stats(self, agent_name: str) -> Dict:
        """Get performance statistics for an agent"""
        results = qdrant_client.scroll(
            collection_name=COLLECTION_AGENT_LEARNING,
            scroll_filter=Filter(
                must=[
                    FieldCondition(
                        key="agent_name",
                        match=MatchValue(value=agent_name)
                    )
                ]
            ),
            limit=100
        )
        
        if not results[0]:
            return {
                "total_experiences": 0,
                "avg_outcome": 0,
                "success_rate": 0
            }
        
        outcomes = [point.payload.get("outcome_score", 0) for point in results[0]]
        
        return {
            "total_experiences": len(outcomes),
            "avg_outcome": np.mean(outcomes) if outcomes else 0,
            "success_rate": sum(1 for o in outcomes if o >= 0.7) / len(outcomes) if outcomes else 0
        }

learning_system = AgentLearningSystem()
print("Agent Learning System initialized")
print("Agents can now store experiences and learn from past successes")

## Step 11: Privacy & Ethics Systems

Responsible AI systems for educational context.

In [ ]:
class PrivacyManager:
    """Ensure student data privacy and consent"""
    
    def __init__(self):
        self.anonymization_enabled = True
    
    def anonymize_student_id(self, student_id: str) -> str:
        """Generate anonymous ID for external sharing"""
        return hashlib.sha256(student_id.encode()).hexdigest()[:16]
    
    def filter_sensitive_data(self, data: Dict) -> Dict:
        """Remove sensitive information from data"""
        filtered = data.copy()
        sensitive_keys = ["student_id", "email", "name"]
        for key in sensitive_keys:
            if key in filtered:
                filtered[key] = self.anonymize_student_id(str(filtered[key]))
        return filtered

class ConsentManager:
    """Track student consent for data usage"""
    
    def __init__(self):
        self.consent_records = {}
    
    def record_consent(self, student_id: str, consent_type: str, granted: bool):
        """Record student consent"""
        if student_id not in self.consent_records:
            self.consent_records[student_id] = {}
        self.consent_records[student_id][consent_type] = {
            "granted": granted,
            "timestamp": datetime.now().isoformat()
        }
    
    def has_consent(self, student_id: str, consent_type: str) -> bool:
        """Check if student has granted consent"""
        return self.consent_records.get(student_id, {}).get(consent_type, {}).get("granted", False)

class BiasAuditor:
    """Monitor and mitigate bias in teaching"""
    
    def __init__(self):
        self.interaction_log = []
    
    def log_interaction(self, student_profile: Dict, content_delivered: Dict):
        """Log interaction for bias analysis"""
        self.interaction_log.append({
            "timestamp": datetime.now().isoformat(),
            "student_demographics": student_profile.get("demographics", {}),
            "content": content_delivered,
            "difficulty": content_delivered.get("difficulty")
        })
    
    def analyze_bias(self) -> Dict:
        """Analyze logged interactions for bias patterns"""
        if not self.interaction_log:
            return {"status": "insufficient_data"}
        
        return {
            "total_interactions": len(self.interaction_log),
            "status": "monitored",
            "recommendation": "Continue monitoring for patterns"
        }

class ExplainabilityEngine:
    """Provide explanations for AI decisions"""
    
    def explain_recommendation(self, recommendation: Dict, context: Dict) -> str:
        """Generate human-readable explanation for recommendations"""
        if recommendation.get("type") == "review":
            return f"Recommending review of '{recommendation['recommendation']}' because the student showed confusion during the last session."
        elif recommendation.get("type") == "next":
            return f"Student has mastered {len(context.get('mastered_topics', []))} topics. Moving to next topic in sequence."
        else:
            return "Starting with foundational content for new student."

privacy_manager = PrivacyManager()
consent_manager = ConsentManager()
bias_auditor = BiasAuditor()
explainability_engine = ExplainabilityEngine()

print("Privacy & Ethics systems initialized")
print("All data handling will respect consent and privacy")

## Step 12: OpenAI Swarm Agents

Now we define the 6 specialized agents. All dependencies are already in place.

In [ ]:
def orchestrator_agent_logic(context_variables: Dict):
    """
    Master agent that coordinates all other agents
    Decides which agent should handle the student's current state
    """
    student_message = context_variables.get("student_message", "")
    emotion_data = detect_emotion(student_message)
    dominant_emotion = emotion_data["dominant"]
    
    agent_context["current_emotion"] = dominant_emotion
    agent_context["conversation_history"].append({
        "message": student_message,
        "emotion": dominant_emotion,
        "timestamp": datetime.now().isoformat()
    })
    
    # Retrieve successful strategies for similar emotional states
    past_strategies = learning_system.retrieve_successful_strategies(
        agent_name="orchestrator",
        situation=f"Student emotion: {dominant_emotion}",
        top_k=2
    )
    
    strategy_context = ""
    if past_strategies:
        strategy_context = f"\nPrevious successful approaches: {past_strategies[0]['action']}"
    
    decision = f"""Analyze the situation and decide next action:
- Student emotion: {dominant_emotion}
- Message: {student_message[:100]}
{strategy_context}

Choose appropriate agent: content_sequencer, emotion_analyzer, style_adapter, explainer, or memory_manager"""
    
    return Result(value=decision, agent=content_sequencer_agent)

orchestrator_agent = Agent(
    name="Orchestrator",
    instructions="""You are the master coordinator of the teaching system.
Analyze student state and route to appropriate specialized agent.
Consider emotion, progress, and past successful strategies.""",
    functions=[orchestrator_agent_logic]
)

def get_next_content(context_variables: Dict):
    """
    Content Sequencer agent - decides what to teach next
    Uses student memory and learning system for informed decisions
    """
    student_id = context_variables.get("student_id")
    
    if not student_id:
        return Result(value="Please provide student ID to continue")
    
    # Get student progress
    student_profile = memory_system.get_student_profile(student_id)
    
    # Get past successful content sequencing strategies
    past_strategies = learning_system.retrieve_successful_strategies(
        agent_name="content_sequencer",
        situation=f"Student progress: {len(student_profile.get('mastered_topics', []))} topics mastered",
        top_k=3
    )
    
    # Get recommendation
    next_topic = memory_system.recommend_next_topic(student_id)
    
    # Retrieve actual content from PDF
    content = query_pdf_content(sequence_id=next_topic.get("sequence_id", 0))
    
    strategy_notes = ""
    if past_strategies:
        strategy_notes = f"\nLearned approach: {past_strategies[0]['action']}"
    
    result = f"""Next content recommendation:
Topic: {next_topic['recommendation']}
Reason: {next_topic['reason']}
Content: {content.get('text', 'No content available')[:200]}
{strategy_notes}"""
    
    # Store this decision for learning
    learning_system.store_experience(
        agent_name="content_sequencer",
        situation=f"Sequenced content for student with {len(student_profile.get('topics_covered', []))} topics covered",
        action=f"Recommended {next_topic['type']} content: {next_topic['recommendation']}",
        outcome_score=0.8,  # Will be updated based on student performance
        metadata={"topic": next_topic['recommendation'], "type": next_topic['type']}
    )
    
    return Result(value=result)

content_sequencer_agent = Agent(
    name="ContentSequencer",
    instructions="""You determine what content to teach next based on student progress.
Use student memory to avoid repetition and focus on knowledge gaps.
Apply learned sequencing strategies from past successful sessions.""",
    functions=[get_next_content]
)

def analyze_student_emotion(context_variables: Dict):
    """
    Emotion Analyzer agent - provides deep emotional insights
    """
    student_message = context_variables.get("student_message", "")
    emotion_data = detect_emotion(student_message)
    
    # Query appropriate teaching style
    teaching_style = query_teaching_style(emotion_data["dominant"])
    
    # Get past successful emotional handling strategies
    past_strategies = learning_system.retrieve_successful_strategies(
        agent_name="emotion_analyzer",
        situation=f"Student showing {emotion_data['dominant']} emotion",
        top_k=2
    )
    
    strategy_guidance = ""
    if past_strategies:
        strategy_guidance = f"\nProven approach: {past_strategies[0]['action']}"
    
    analysis = f"""Emotional Analysis:
Dominant emotion: {emotion_data['dominant']}
All scores: {emotion_data['scores']}

Recommended teaching style: {teaching_style['name']}
Style characteristics: {teaching_style.get('characteristics', 'Standard approach')}
{strategy_guidance}

Adjust tone and pacing accordingly."""
    
    # Store emotion handling experience
    learning_system.store_experience(
        agent_name="emotion_analyzer",
        situation=f"Detected {emotion_data['dominant']} emotion",
        action=f"Applied {teaching_style['name']} teaching style",
        outcome_score=0.75,
        metadata={"emotion": emotion_data['dominant'], "style": teaching_style['name']}
    )
    
    return Result(value=analysis)

emotion_analyzer_agent = Agent(
    name="EmotionAnalyzer",
    instructions="""You analyze student emotions and recommend appropriate teaching approaches.
Detect frustration, confusion, confidence, and adjust teaching style accordingly.
Learn from past successful emotional interventions.""",
    functions=[analyze_student_emotion]
)

def adapt_teaching_style(context_variables: Dict):
    """
    Style Adapter agent - modifies content delivery style
    """
    current_emotion = agent_context.get("current_emotion", "neutral")
    current_topic = agent_context.get("current_topic", "general")
    
    # Get past successful style adaptations
    past_strategies = learning_system.retrieve_successful_strategies(
        agent_name="style_adapter",
        situation=f"Teaching {current_topic} to student feeling {current_emotion}",
        top_k=2
    )
    
    teaching_style = query_teaching_style(current_emotion)
    
    adaptation = f"""Style Adaptation for {current_emotion} student:

Teaching style: {teaching_style['name']}
Approach: {teaching_style.get('description', 'Balanced approach')}

Delivery modifications:
- Tone: {'Encouraging and supportive' if current_emotion in ['fear', 'sadness'] else 'Enthusiastic and engaging'}
- Pace: {'Slower with more examples' if current_emotion == 'fear' else 'Moderate'}
- Complexity: {'Simplified concepts first' if current_emotion == 'fear' else 'Progressive complexity'}
"""
    
    if past_strategies:
        adaptation += f"\nLearned from past: {past_strategies[0]['action']}"
    
    # Store adaptation decision
    learning_system.store_experience(
        agent_name="style_adapter",
        situation=f"Adapted style for {current_emotion} emotion",
        action=f"Used {teaching_style['name']} approach",
        outcome_score=0.8,
        metadata={"emotion": current_emotion, "style": teaching_style['name']}
    )
    
    return Result(value=adaptation)

style_adapter_agent = Agent(
    name="StyleAdapter",
    instructions="""You modify teaching delivery style based on student's emotional state.
Adapt tone, pacing, complexity, and examples to match student needs.
Apply successful adaptation strategies from agent learning system.""",
    functions=[adapt_teaching_style]
)

def explain_concept(context_variables: Dict):
    """
    Explainer agent - provides detailed explanations with examples
    """
    concept = context_variables.get("concept", "")
    difficulty_level = context_variables.get("difficulty", "beginner")
    
    # Use Groq for detailed explanation
    try:
        response = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": f"""You are an expert teacher explaining concepts at {difficulty_level} level.
Provide clear explanations with practical examples.
Break down complex ideas into understandable parts."""
                },
                {
                    "role": "user",
                    "content": f"Explain this concept clearly: {concept}"
                }
            ],
            temperature=0.7,
            max_tokens=500
        )
        
        explanation = response.choices[0].message.content
        
        # Store successful explanation
        learning_system.store_experience(
            agent_name="explainer",
            situation=f"Explained {concept} at {difficulty_level} level",
            action=f"Generated detailed explanation",
            outcome_score=0.85,
            metadata={"concept": concept, "difficulty": difficulty_level}
        )
        
        return Result(value=explanation)
        
    except Exception as e:
        return Result(value=f"Could not generate explanation: {str(e)}")

explainer_agent = Agent(
    name="Explainer",
    instructions="""You provide detailed, clear explanations of concepts.
Use analogies, examples, and step-by-step breakdowns.
Adapt explanation depth to student's current understanding level.""",
    functions=[explain_concept]
)

def manage_student_memory(context_variables: Dict):
    """
    Memory Manager agent - tracks and updates student progress
    """
    student_id = context_variables.get("student_id")
    action = context_variables.get("action", "get_profile")
    
    if action == "get_profile":
        profile = memory_system.get_student_profile(student_id)
        if profile:
            return Result(value=f"""Student Profile:
Topics covered: {len(profile.get('topics_covered', []))}
Mastered topics: {profile.get('mastered_topics', [])}
Confusion points: {profile.get('confusion_points', [])}
Total interactions: {profile.get('total_interactions', 0)}
Last emotion: {profile.get('last_emotion', 'N/A')}""")
        else:
            return Result(value="Student not found. Initializing new student.")
    
    elif action == "update_progress":
        topic = context_variables.get("topic")
        understood = context_variables.get("understood", True)
        emotion = context_variables.get("emotion", "neutral")
        style = context_variables.get("style", "Balanced")
        
        memory_system.update_student_progress(student_id, topic, understood, emotion, style)
        
        # Store memory management action
        learning_system.store_experience(
            agent_name="memory_manager",
            situation=f"Updated progress for topic: {topic}",
            action=f"Recorded {'success' if understood else 'confusion'}",
            outcome_score=1.0 if understood else 0.5,
            metadata={"topic": topic, "understood": understood}
        )
        
        return Result(value=f"Progress updated for {student_id}")
    
    return Result(value="Unknown memory action")

memory_manager_agent = Agent(
    name="MemoryManager",
    instructions="""You track student learning progress and preferences.
Update student profiles after each interaction.
Identify patterns in learning behavior for better recommendations.""",
    functions=[manage_student_memory]
)

print("All 6 agents initialized successfully")
print("Agents: Orchestrator, Content Sequencer, Emotion Analyzer, Style Adapter, Explainer, Memory Manager")

## Step 13: Adaptive Tutoring Session

Basic teaching session that uses all agents collaboratively.

In [ ]:
class AdaptiveTutoringSession:
    """
    Main teaching session orchestrating all agents
    """
    
    def __init__(self, student_id: str):
        self.student_id = student_id
        self.client = Swarm()
        self.session_log = []
        self.session_start = datetime.now()
        
        # Initialize student if needed
        profile = memory_system.get_student_profile(student_id)
        if not profile:
            memory_system.initialize_student(student_id)
            print(f"New student initialized: {student_id}")
        
        # Set up agent context
        agent_context["current_student"] = student_id
    
    def start_session(self, initial_message: str = "I'm ready to learn"):
        """Begin a teaching session"""
        print(f"\n=== Starting Adaptive Tutoring Session for {self.student_id} ===\n")
        
        context = {
            "student_id": self.student_id,
            "student_message": initial_message
        }
        
        # Run orchestrator
        response = self.client.run(
            agent=orchestrator_agent,
            messages=[{"role": "user", "content": initial_message}],
            context_variables=context
        )
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "session_start",
            "message": initial_message,
            "response": response.messages[-1]["content"] if response.messages else "No response"
        })
        
        return response.messages[-1]["content"] if response.messages else "Session initialized"
    
    def get_next_lesson(self):
        """Get next content to teach"""
        print(f"\nFetching next lesson for {self.student_id}...")
        
        context = {"student_id": self.student_id}
        
        response = self.client.run(
            agent=content_sequencer_agent,
            messages=[{"role": "user", "content": "What should I learn next?"}],
            context_variables=context
        )
        
        content = response.messages[-1]["content"] if response.messages else "No content available"
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "get_next_lesson",
            "content": content
        })
        
        print(f"\n{content}")
        return content
    
    def student_response(self, message: str):
        """Process student response during teaching"""
        print(f"\nStudent: {message}")
        
        # Detect emotion
        emotion = detect_emotion(message)
        print(f"Detected emotion: {emotion['dominant']}")
        
        context = {
            "student_id": self.student_id,
            "student_message": message
        }
        
        # Analyze emotion
        emotion_response = self.client.run(
            agent=emotion_analyzer_agent,
            messages=[{"role": "user", "content": message}],
            context_variables=context
        )
        
        emotion_analysis = emotion_response.messages[-1]["content"] if emotion_response.messages else ""
        print(f"\nEmotion Analysis:\n{emotion_analysis}")
        
        # Adapt style
        style_response = self.client.run(
            agent=style_adapter_agent,
            messages=[{"role": "user", "content": "How should I adapt my teaching?"}],
            context_variables=context
        )
        
        style_adaptation = style_response.messages[-1]["content"] if style_response.messages else ""
        print(f"\nStyle Adaptation:\n{style_adaptation}")
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "student_response",
            "message": message,
            "emotion": emotion,
            "emotion_analysis": emotion_analysis,
            "style_adaptation": style_adaptation
        })
        
        return {
            "emotion": emotion,
            "analysis": emotion_analysis,
            "adaptation": style_adaptation
        }
    
    def explain(self, concept: str):
        """Get detailed explanation of concept"""
        print(f"\nExplaining: {concept}")
        
        context = {
            "concept": concept,
            "difficulty": "intermediate"
        }
        
        response = self.client.run(
            agent=explainer_agent,
            messages=[{"role": "user", "content": f"Explain: {concept}"}],
            context_variables=context
        )
        
        explanation = response.messages[-1]["content"] if response.messages else "No explanation available"
        
        print(f"\n{explanation}")
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "explain",
            "concept": concept,
            "explanation": explanation
        })
        
        return explanation
    
    def update_progress(self, topic: str, understood: bool):
        """Update student progress after teaching a topic"""
        current_emotion = agent_context.get("current_emotion", "neutral")
        
        context = {
            "student_id": self.student_id,
            "action": "update_progress",
            "topic": topic,
            "understood": understood,
            "emotion": current_emotion,
            "style": "Adaptive"
        }
        
        response = self.client.run(
            agent=memory_manager_agent,
            messages=[{"role": "user", "content": f"Update progress for {topic}"}],
            context_variables=context
        )
        
        print(f"\nProgress updated: {topic} - {'Mastered' if understood else 'Needs review'}")
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "update_progress",
            "topic": topic,
            "understood": understood
        })
    
    def end_session(self):
        """End teaching session and generate report"""
        session_duration = (datetime.now() - self.session_start).total_seconds()
        
        # Get final student profile
        profile = memory_system.get_student_profile(self.student_id)
        
        report = {
            "student_id": self.student_id,
            "session_duration_seconds": session_duration,
            "total_interactions": len(self.session_log),
            "topics_covered": len([log for log in self.session_log if log.get("action") == "get_next_lesson"]),
            "student_profile": profile,
            "session_log": self.session_log
        }
        
        print(f"\n=== Session Complete ===")
        print(f"Duration: {session_duration:.1f} seconds")
        print(f"Interactions: {len(self.session_log)}")
        print(f"Topics covered: {report['topics_covered']}")
        
        return report

print("Adaptive Tutoring Session class ready")

## Step 14: Intelligent Lesson Planner

PHASE 1: PDF → Generate Lesson Plan → Iterate → Approve

In [ ]:
class IntelligentLessonPlanner:
    """
    Phase 1: Generate and iterate on lesson plans before teaching begins
    Human-in-the-loop lesson planning
    """
    
    def __init__(self):
        self.current_plan = None
        self.iteration_history = []
    
    def analyze_pdf_content(self, sample_size: int = 200) -> Dict:
        """
        Analyze all ingested PDF content to extract actual structure and topics
        """
        print("\nAnalyzing PDF content structure...")
        
        # Get content from Qdrant
        results = qdrant_client.scroll(
            collection_name=COLLECTION_PDF_CONTENT,
            limit=sample_size
        )
        
        if not results[0]:
            return {
                "status": "error",
                "message": "No PDF content found. Please upload a PDF first."
            }
        
        all_content = []
        content_samples = []
        
        for point in results[0]:
            payload = point.payload
            all_content.append({
                "sequence_id": payload.get("sequence_id"),
                "text": payload.get("text", ""),
                "page": payload.get("page"),
                "difficulty": payload.get("difficulty", "intermediate")
            })
            
            # Collect samples for chapter detection
            if len(content_samples) < 50:
                content_samples.append(payload.get("text", "")[:300])
        
        # Combine samples for LLM analysis
        combined_samples = "\n\n---\n\n".join(content_samples[:20])
        
        analysis = {
            "total_sections": len(all_content),
            "difficulty_levels": list(set(item["difficulty"] for item in all_content)),
            "content_preview": all_content[:10],
            "content_samples": combined_samples
        }
        
        print(f"Found {len(all_content)} content sections")
        print(f"Extracted samples from first 20 sections for analysis")
        
        return analysis
    
    def generate_lesson_plan(self, learning_objectives: str = None, 
                           target_duration: int = 60,
                           difficulty_preference: str = "progressive") -> Dict:
        """
        Generate initial lesson plan using Groq LLM
        """
        print(f"\nGenerating lesson plan...")
        print(f"Duration: {target_duration} minutes")
        print(f"Difficulty: {difficulty_preference}")
        
        # Analyze PDF content
        content_analysis = self.analyze_pdf_content()
        
        if content_analysis.get("status") == "error":
            return content_analysis
        
        # Create prompt for Groq
        prompt = f"""You are analyzing a PDF document to create a detailed teaching curriculum.

ACTUAL PDF CONTENT SAMPLES:
{content_analysis['content_samples']}

Based on the actual content above, create a DETAILED lesson plan that follows the PDF structure.

INSTRUCTIONS:
1. Identify actual chapters/sections from the PDF content
2. For EACH major topic, create lessons with subsections (1, 1.1, 1.2, etc.)
3. After EACH major lesson, include a quiz checkpoint
4. Extract real topic names from the PDF (not generic names)
5. Total available sections: {content_analysis['total_sections']}

Requirements:
- Target duration: {target_duration} minutes per major lesson
- Create lessons for at least 8-10 major topics from the PDF
- Each lesson should have 2-5 subsections
- Include quiz after each major lesson

Output JSON format:
{{
  "learningObjectives": ["objective1", "objective2", ...],
  "lessons": [
    {{
      "lessonNumber": 1,
      "title": "Actual Chapter Title from PDF",
      "duration": 15,
      "subsections": [
        {{"number": "1.1", "title": "Subsection from PDF", "sequenceStart": 0, "sequenceEnd": 10}},
        {{"number": "1.2", "title": "Next subsection", "sequenceStart": 11, "sequenceEnd": 20}}
      ],
      "teachingStrategy": "How to teach this",
      "quizAfter": true
    }},
    {{
      "lessonNumber": 2,
      "title": "Next Chapter Title",
      ...
    }}
  ]
}}

IMPORTANT: Base everything on the ACTUAL PDF content shown above, not generic topics."""
        
        try:
            response = groq_client.chat.completions.create(
                model=GROQ_MODEL,
                messages=[
                    {
                        "role": "system",
                        "content": "You are an expert curriculum designer creating structured lesson plans. Output valid JSON only."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=0.8,
                max_tokens=2000
            )
            
            plan_text = response.choices[0].message.content
            
            # Extract JSON from markdown code blocks if present
            import re
            json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', plan_text, re.DOTALL)
            if json_match:
                plan_text = json_match.group(1)
            
            # Try to parse as JSON, fallback to text
            try:
                plan_data = json.loads(plan_text)
            except Exception as parse_error:
                plan_data = {
                    "plan_text": plan_text,
                    "structured": False,
                    "parse_error": str(parse_error)
                }
            
            self.current_plan = {
                "version": 1,
                "created_at": datetime.now().isoformat(),
                "parameters": {
                    "learning_objectives": learning_objectives,
                    "target_duration": target_duration,
                    "difficulty_preference": difficulty_preference
                },
                "content_analysis": content_analysis,
                "plan": plan_data
            }
            
            self.iteration_history.append({
                "version": 1,
                "action": "initial_generation",
                "timestamp": datetime.now().isoformat()
            })
            
            print("\nLesson plan generated successfully!")
            return self.current_plan
            
        except Exception as e:
            return {
                "status": "error",
                "message": f"Failed to generate lesson plan: {str(e)}"
            }
    
    def iterate_plan(self, feedback: str) -> Dict:
        """
        Refine lesson plan based on human feedback
        Human-in-the-loop iteration
        """
        if not self.current_plan:
            return {
                "status": "error",
                "message": "No lesson plan exists. Generate one first."
            }
        
        print(f"\nIterating on lesson plan based on feedback...")
        print(f"Feedback: {feedback[:100]}...")
        
        current_version = self.current_plan["version"]
        
        prompt = f"""You are refining a lesson plan based on feedback.

Current Lesson Plan:
{json.dumps(self.current_plan['plan'], indent=2)}

Feedback from instructor:
{feedback}

Create an improved version that addresses the feedback while maintaining structure.
Output as JSON with same structure as before."""
        
        try:
            response = groq_client.chat.completions.create(
                model=GROQ_MODEL,
                messages=[
                    {
                        "role": "system",
                        "content": "You refine lesson plans based on feedback. Output valid JSON only."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=0.7,
                max_tokens=2000
            )
            
            refined_plan_text = response.choices[0].message.content
            
            # Extract JSON from markdown code blocks if present
            import re
            json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', refined_plan_text, re.DOTALL)
            if json_match:
                refined_plan_text = json_match.group(1)
            
            try:
                import json
                refined_plan = json.loads(refined_plan_text)
            except Exception as parse_error:
                refined_plan = {
                    "plan_text": refined_plan_text,
                    "structured": False,
                    "parse_error": str(parse_error)
                }
            
            self.current_plan["version"] = current_version + 1
            self.current_plan["plan"] = refined_plan
            self.current_plan["last_updated"] = datetime.now().isoformat()
            
            self.iteration_history.append({
                "version": current_version + 1,
                "action": "iteration",
                "feedback": feedback,
                "timestamp": datetime.now().isoformat()
            })
            
            print(f"\nLesson plan updated to version {self.current_plan['version']}")
            return self.current_plan
            
        except Exception as e:
            return {
                "status": "error",
                "message": f"Failed to iterate plan: {str(e)}"
            }
    
    def approve_plan(self) -> Dict:
        """
        Finalize and approve lesson plan
        Marks plan as ready for teaching
        """
        if not self.current_plan:
            return {
                "status": "error",
                "message": "No lesson plan to approve"
            }
        
        self.current_plan["approved"] = True
        self.current_plan["approved_at"] = datetime.now().isoformat()
        self.current_plan["final_version"] = self.current_plan["version"]
        
        self.iteration_history.append({
            "version": self.current_plan["version"],
            "action": "approval",
            "timestamp": datetime.now().isoformat()
        })
        
        print("\nLesson plan APPROVED and ready for teaching!")
        print(f"Final version: {self.current_plan['version']}")
        print(f"Total iterations: {len(self.iteration_history)}")
        
        return {
            "status": "approved",
            "plan": self.current_plan,
            "iteration_count": len(self.iteration_history)
        }
    
    def get_current_plan(self) -> Dict:
        """Get current lesson plan"""
        return self.current_plan
    
    def get_iteration_history(self) -> List[Dict]:
        """Get history of all iterations"""
        return self.iteration_history

planner = IntelligentLessonPlanner()
print("Intelligent Lesson Planner ready")
print("Workflow: generate_lesson_plan() to  iterate_plan(feedback) to approve_plan()")

## Step 15: Quiz System

Generate adaptive quizzes to assess understanding.

In [ ]:
class QuizSystem:
    """
    Generate and evaluate quizzes based on taught content
    """
    
    def __init__(self):
        self.quiz_history = []
    
    def generate_quiz(self, topic: str, difficulty: str = "intermediate", 
                     num_questions: int = 3) -> Dict:
        """
        Generate quiz questions using Groq LLM
        """
        print(f"\nGenerating quiz: {topic} ({difficulty})")
        print(f"Questions: {num_questions}")
        
        # Get related content from PDF
        content = query_pdf_content(topic_search=topic, difficulty=difficulty)
        
        prompt = f"""Generate {num_questions} multiple choice questions to test understanding.

Topic: {topic}
Difficulty: {difficulty}
Content context: {content.get('text', '')[:500]}

For each question provide:
1. Question text
2. 4 answer options (A, B, C, D)
3. Correct answer (letter)
4. Explanation of why that's correct

Format as JSON array of questions."""
        
        try:
            response = groq_client.chat.completions.create(
                model=GROQ_MODEL,
                messages=[
                    {
                        "role": "system",
                        "content": "You create educational quiz questions. Output valid JSON only."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=0.7,
                max_tokens=1500
            )
            
            quiz_text = response.choices[0].message.content
            
            try:
                import json
                questions = json.loads(quiz_text)
            except:
                questions = {
                    "questions": [
                        {
                            "question": "Sample question about " + topic,
                            "options": ["A: Option 1", "B: Option 2", "C: Option 3", "D: Option 4"],
                            "correct": "A",
                            "explanation": "This is a fallback question"
                        }
                    ],
                    "parse_error": True
                }
            
            quiz = {
                "quiz_id": str(uuid.uuid4()),
                "topic": topic,
                "difficulty": difficulty,
                "num_questions": num_questions,
                "questions": questions if isinstance(questions, list) else questions.get("questions", []),
                "created_at": datetime.now().isoformat()
            }
            
            self.quiz_history.append(quiz)
            
            print(f"\nQuiz generated with ID: {quiz['quiz_id'][:8]}")
            return quiz
            
        except Exception as e:
            return {
                "status": "error",
                "message": f"Failed to generate quiz: {str(e)}"
            }
    
    def evaluate_answers(self, quiz_id: str, student_answers: Dict[int, str]) -> Dict:
        """
        Evaluate student's quiz answers
        
        Args:
            quiz_id: ID of the quiz
            student_answers: Dict mapping question index to student's answer (e.g., {0: "A", 1: "B"})
        """
        # Find quiz
        quiz = None
        for q in self.quiz_history:
            if q["quiz_id"] == quiz_id:
                quiz = q
                break
        
        if not quiz:
            return {
                "status": "error",
                "message": "Quiz not found"
            }
        
        questions = quiz["questions"]
        results = []
        correct_count = 0
        
        for idx, student_answer in student_answers.items():
            if idx >= len(questions):
                continue
            
            question = questions[idx]
            correct_answer = question.get("correct", "").upper()
            student_answer_clean = student_answer.upper().strip()
            
            is_correct = student_answer_clean == correct_answer
            
            if is_correct:
                correct_count += 1
            
            results.append({
                "question_index": idx,
                "question": question.get("question"),
                "student_answer": student_answer,
                "correct_answer": correct_answer,
                "is_correct": is_correct,
                "explanation": question.get("explanation", "")
            })
        
        total_questions = len(student_answers)
        score_percentage = (correct_count / total_questions * 100) if total_questions > 0 else 0
        
        evaluation = {
            "quiz_id": quiz_id,
            "topic": quiz["topic"],
            "total_questions": total_questions,
            "correct_answers": correct_count,
            "score_percentage": score_percentage,
            "passed": score_percentage >= 70,
            "results": results,
            "evaluated_at": datetime.now().isoformat()
        }
        
        print(f"\nQuiz Results:")
        print(f"Score: {correct_count}/{total_questions} ({score_percentage:.1f}%)")
        print(f"Status: {'PASSED' if evaluation['passed'] else 'NEEDS REVIEW'}")
        
        return evaluation
    
    def get_quiz_by_id(self, quiz_id: str) -> Dict:
        """Retrieve quiz by ID"""
        for quiz in self.quiz_history:
            if quiz["quiz_id"] == quiz_id:
                return quiz
        return None

quiz_system = QuizSystem()
print("Quiz System initialized")
print("Can generate adaptive quizzes and evaluate answers")

## Step 16: Enhanced Teaching Session V2

Complete teaching session with whiteboard and quiz integration.

In [ ]:
class AdaptiveTutoringSessionV2:
    """
    Enhanced teaching session with whiteboard images and quiz checkpoints
    Full-featured teaching experience
    """
    
    def __init__(self, student_id: str, lesson_plan: Dict = None):
        self.student_id = student_id
        self.lesson_plan = lesson_plan
        self.client = Swarm()
        self.session_log = []
        self.session_start = datetime.now()
        self.current_topic = None
        self.whiteboard_images = []
        self.quiz_results = []
        
        # Initialize student
        profile = memory_system.get_student_profile(student_id)
        if not profile:
            memory_system.initialize_student(student_id)
            print(f"New student initialized: {student_id}")
        
        agent_context["current_student"] = student_id
        
        print(f"\n=== Enhanced Adaptive Tutoring Session ===")
        print(f"Student: {student_id}")
        if lesson_plan:
            print(f"Following approved lesson plan v{lesson_plan.get('version')}")
    
    def teach_topic(self, topic: str, sequence_id: int = None):
        """
        Teach a specific topic with full adaptive features
        """
        print(f"\n--- Teaching Topic: {topic} ---")
        self.current_topic = topic
        agent_context["current_topic"] = topic
        
        # Get content
        content = query_pdf_content(sequence_id=sequence_id, topic_search=topic)
        
        print(f"\nContent: {content.get('text', 'No content')[:300]}...")
        
        # Get whiteboard images if available
        if content.get("page"):
            images = query_related_images(content["page"])
            if images:
                print(f"\nWhiteboard: {len(images)} visual aids available")
                self.whiteboard_images.extend(images)
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "teach_topic",
            "topic": topic,
            "content": content.get("text", "")[:500]
        })
        
        return content
    
    def process_student_input(self, message: str):
        """
        Process student message with full emotion and style adaptation
        """
        print(f"\nStudent: {message}")
        
        # Emotion detection
        emotion = detect_emotion(message)
        print(f"Emotion detected: {emotion['dominant']}")
        
        context = {
            "student_id": self.student_id,
            "student_message": message
        }
        
        # Get emotion analysis
        emotion_response = self.client.run(
            agent=emotion_analyzer_agent,
            messages=[{"role": "user", "content": message}],
            context_variables=context
        )
        
        # Adapt teaching style
        style_response = self.client.run(
            agent=style_adapter_agent,
            messages=[{"role": "user", "content": "Adapt teaching style"}],
            context_variables=context
        )
        
        # Generate response
        response_context = {
            "concept": self.current_topic or "current topic",
            "difficulty": "intermediate"
        }
        
        explanation_response = self.client.run(
            agent=explainer_agent,
            messages=[{"role": "user", "content": f"Respond to: {message}"}],
            context_variables=response_context
        )
        
        response_text = explanation_response.messages[-1]["content"] if explanation_response.messages else "I understand your question."
        
        print(f"\nTeacher: {response_text[:200]}...")
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "student_interaction",
            "student_message": message,
            "emotion": emotion,
            "teacher_response": response_text
        })
        
        return {
            "emotion": emotion,
            "response": response_text
        }
    
    def checkpoint_quiz(self, topic: str, num_questions: int = 3):
        """
        Administer checkpoint quiz
        """
        print(f"\n=== Quiz Checkpoint: {topic} ===")
        
        # Generate quiz
        quiz = quiz_system.generate_quiz(topic, difficulty="intermediate", num_questions=num_questions)
        
        if quiz.get("status") == "error":
            print(f"Quiz generation failed: {quiz.get('message')}")
            return None
        
        # Display questions
        print("\nQuiz Questions:")
        questions = quiz.get("questions", [])
        for idx, q in enumerate(questions):
            print(f"\nQ{idx + 1}: {q.get('question', 'No question')}")
            for option in q.get("options", []):
                print(f"  {option}")
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "quiz_checkpoint",
            "topic": topic,
            "quiz_id": quiz.get("quiz_id")
        })
        
        return quiz
    
    def submit_quiz_answers(self, quiz_id: str, answers: Dict[int, str]):
        """
        Submit and evaluate quiz answers
        """
        print(f"\nEvaluating quiz answers...")
        
        evaluation = quiz_system.evaluate_answers(quiz_id, answers)
        
        if evaluation.get("status") == "error":
            print(f"Evaluation failed: {evaluation.get('message')}")
            return None
        
        # Update student progress based on quiz performance
        passed = evaluation.get("passed", False)
        self.update_progress(evaluation["topic"], understood=passed)
        
        self.quiz_results.append(evaluation)
        
        self.session_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "quiz_evaluated",
            "quiz_id": quiz_id,
            "score": evaluation.get("score_percentage"),
            "passed": passed
        })
        
        return evaluation
    
    def update_progress(self, topic: str, understood: bool):
        """Update student learning progress"""
        current_emotion = agent_context.get("current_emotion", "neutral")
        
        context = {
            "student_id": self.student_id,
            "action": "update_progress",
            "topic": topic,
            "understood": understood,
            "emotion": current_emotion,
            "style": "Adaptive"
        }
        
        self.client.run(
            agent=memory_manager_agent,
            messages=[{"role": "user", "content": f"Update progress for {topic}"}],
            context_variables=context
        )
        
        print(f"Progress updated: {topic} - {'Mastered' if understood else 'Needs review'}")
    
    def generate_session_report(self) -> Dict:
        """
        Generate comprehensive session report
        """
        session_duration = (datetime.now() - self.session_start).total_seconds()
        
        profile = memory_system.get_student_profile(self.student_id)
        
        report = {
            "student_id": self.student_id,
            "session_start": self.session_start.isoformat(),
            "session_duration_seconds": session_duration,
            "lesson_plan_used": self.lesson_plan is not None,
            "total_interactions": len(self.session_log),
            "topics_taught": len([log for log in self.session_log if log.get("action") == "teach_topic"]),
            "quizzes_taken": len(self.quiz_results),
            "average_quiz_score": np.mean([r["score_percentage"] for r in self.quiz_results]) if self.quiz_results else 0,
            "whiteboard_images_shown": len(self.whiteboard_images),
            "student_profile": profile,
            "quiz_results": self.quiz_results,
            "session_log": self.session_log
        }
        
        print(f"\n=== Session Report ===")
        print(f"Duration: {session_duration / 60:.1f} minutes")
        print(f"Topics taught: {report['topics_taught']}")
        print(f"Quizzes: {report['quizzes_taken']}")
        if self.quiz_results:
            print(f"Average score: {report['average_quiz_score']:.1f}%")
        print(f"Total interactions: {report['total_interactions']}")
        
        # Save report to file
        report_filename = f"session_report_{self.student_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(report_filename, 'w') as f:
            json.dump(report, f, indent=2)
        print(f"\nReport saved: {report_filename}")
        
        return report

print("Enhanced Tutoring Session V2 ready")
print("Features: Whiteboard images, quiz checkpoints, full emotion adaptation")

## Step 17: Test PDF Processing and Improved Lesson Plan Display

Testing with Rich Dad Poor Dad.pdf and displaying lesson plans in hierarchical format.

In [ ]:
# ============================================================================
# COMPLETE TEST INSTRUCTIONS - Read This First!
# ============================================================================

print("="*80)
print("HOW TO RUN THE COMPLETE TEST")
print("="*80)
print("\nOption 1: Run All Cells (RECOMMENDED)")
print("  - Go to menu: Kernel > Restart & Run All")
print("  - This will initialize everything and run all tests")
print("\nOption 2: Manual Step-by-Step")
print("  1. Run cells 3-18 to initialize all systems")
print("  2. Run cell 36 (hierarchical display function)")
print("  3. Run cells below for testing:")
print("     - Test 1: PDF Processing")
print("     - Test 2: Lesson Plan Generation") 
print("     - Test 3: Display Hierarchical Plan")
print("     - Test 4: Student Session")
print("\nCurrent PDF: Rich Dad Poor Dad.pdf")
print("Location: c:\\Users\\dhanu\\OneDrive\\Desktop\\last MAS\\")
print("="*80)

In [ ]:
def display_lesson_plan_hierarchical(lesson_plan: Dict):
    """
    Display lesson plan in hierarchical format with proper numbering
    (1, 1.1, 1.1.1, 2, 2.1, etc.)
    """
    if not lesson_plan:
        print("No lesson plan to display")
        return
    
    print("\n" + "="*80)
    print("LESSON PLAN - HIERARCHICAL VIEW")
    print("="*80)
    
    # Basic info
    print(f"\nVersion: {lesson_plan.get('version', 'N/A')}")
    print(f"Created: {lesson_plan.get('created_at', 'N/A')}")
    if lesson_plan.get('approved'):
        print(f"Status: APPROVED")
    else:
        print(f"Status: DRAFT")
    
    # Parameters
    params = lesson_plan.get('parameters', {})
    if params:
        print(f"\nTarget Duration: {params.get('target_duration', 'N/A')} minutes")
        print(f"Difficulty: {params.get('difficulty_preference', 'N/A')}")
        if params.get('learning_objectives'):
            print(f"Objectives: {params['learning_objectives']}")
    
    print("\n" + "-"*80)
    
    # Get plan content
    plan_content = lesson_plan.get('plan', {})
    
    # Handle nested lessonPlan structure
    if isinstance(plan_content, dict) and 'lessonPlan' in plan_content:
        plan_content = plan_content['lessonPlan']
    
    # If it's structured JSON, parse it
    if isinstance(plan_content, dict):
        # Learning objectives
        objectives_key = None
        for key in ['learning_objectives', 'learningObjectives', 'objectives']:
            if key in plan_content:
                objectives_key = key
                break
        
        if objectives_key:
            objectives = plan_content.get(objectives_key, [])
            if objectives:
                print("\n1. LEARNING OBJECTIVES")
                if isinstance(objectives, list):
                    for idx, obj in enumerate(objectives, 1):
                        print(f"   1.{idx} {obj}")
                else:
                    print(f"   {objectives}")
        
        # Lessons structure (new format)
        section_num = 2
        if 'lessons' in plan_content:
            lessons = plan_content.get('lessons', [])
            if lessons:
                print(f"\n{section_num}. LESSON STRUCTURE")
                for lesson in lessons:
                    lesson_num = lesson.get('lessonNumber') or lesson.get('lesson_number', '?')
                    title = lesson.get('title') or lesson.get('name', 'Untitled Lesson')
                    duration = lesson.get('duration', '')
                    
                    print(f"\n   {section_num}.{lesson_num} {title}")
                    if duration:
                        print(f"        Duration: {duration} minutes")
                    
                    # Subsections
                    subsections = lesson.get('subsections', [])
                    if subsections:
                        for subsection in subsections:
                            sub_num = subsection.get('number', '')
                            sub_title = subsection.get('title', '')
                            seq_range = ""
                            if 'sequenceStart' in subsection and 'sequenceEnd' in subsection:
                                seq_range = f" (sections {subsection['sequenceStart']}-{subsection['sequenceEnd']})"
                            print(f"        {section_num}.{lesson_num}.{sub_num.split('.')[-1] if '.' in sub_num else sub_num} {sub_title}{seq_range}")
                    
                    # Teaching strategy
                    strategy = lesson.get('teachingStrategy') or lesson.get('strategy', '')
                    if strategy:
                        print(f"        Strategy: {strategy[:100]}...")
                    
                    # Quiz checkpoint
                    if lesson.get('quizAfter') or lesson.get('quiz_after'):
                        print(f"        >> QUIZ CHECKPOINT after this lesson")
                
                section_num += 1
        
        # Content sequence / Topics (old format - fallback)
        elif ('content_sequence' in plan_content or 'contentSequence' in plan_content or 
            'topics' in plan_content or 'sections' in plan_content):
            topics = (plan_content.get('content_sequence') or 
                     plan_content.get('contentSequence') or 
                     plan_content.get('topics') or 
                     plan_content.get('sections', []))
            
            if topics:
                print(f"\n{section_num}. CONTENT SEQUENCE")
                if isinstance(topics, list):
                    for idx, topic in enumerate(topics, 1):
                        if isinstance(topic, dict):
                            topic_name = (topic.get('name') or topic.get('topic') or 
                                        topic.get('title') or topic.get('section', f'Topic {idx}'))
                            duration = topic.get('duration') or topic.get('time') or topic.get('estimatedTime', '')
                            strategy = topic.get('teachingStrategy') or topic.get('strategy', '')
                            
                            print(f"   {section_num}.{idx} {topic_name}")
                            if duration:
                                print(f"        Duration: {duration}")
                            if strategy:
                                print(f"        Strategy: {strategy[:100]}...")
                            
                            # Subtopics if any
                            subtopics = topic.get('subtopics') or topic.get('content', [])
                            if subtopics and isinstance(subtopics, list):
                                for sub_idx, subtopic in enumerate(subtopics, 1):
                                    if isinstance(subtopic, str):
                                        print(f"        {section_num}.{idx}.{sub_idx} {subtopic}")
                                    elif isinstance(subtopic, dict):
                                        sub_name = subtopic.get('name') or subtopic.get('title', f'Subtopic {sub_idx}')
                                        print(f"        {section_num}.{idx}.{sub_idx} {sub_name}")
                        else:
                            print(f"   {section_num}.{idx} {topic}")
                section_num += 1
        
        # Teaching strategies
        if 'teaching_strategies' in plan_content or 'strategies' in plan_content:
            strategies = plan_content.get('teaching_strategies') or plan_content.get('strategies', [])
            if strategies:
                print(f"\n{section_num}. TEACHING STRATEGIES")
                if isinstance(strategies, list):
                    for idx, strategy in enumerate(strategies, 1):
                        if isinstance(strategy, dict):
                            strat_name = strategy.get('name') or strategy.get('strategy', f'Strategy {idx}')
                            strat_desc = strategy.get('description') or strategy.get('details', '')
                            print(f"   {section_num}.{idx} {strat_name}")
                            if strat_desc:
                                print(f"        {strat_desc}")
                        else:
                            print(f"   {section_num}.{idx} {strategy}")
                elif isinstance(strategies, dict):
                    for idx, (key, value) in enumerate(strategies.items(), 1):
                        print(f"   {section_num}.{idx} {key}: {value}")
                section_num += 1
        
        # Assessment checkpoints
        if 'assessment' in plan_content or 'assessments' in plan_content or 'checkpoints' in plan_content:
            assessments = (plan_content.get('assessment') or 
                          plan_content.get('assessments') or 
                          plan_content.get('checkpoints', []))
            if assessments:
                print(f"\n{section_num}. ASSESSMENT CHECKPOINTS")
                if isinstance(assessments, list):
                    for idx, assessment in enumerate(assessments, 1):
                        if isinstance(assessment, dict):
                            assess_name = assessment.get('name') or assessment.get('type', f'Checkpoint {idx}')
                            assess_timing = assessment.get('timing') or assessment.get('when', '')
                            print(f"   {section_num}.{idx} {assess_name}")
                            if assess_timing:
                                print(f"        Timing: {assess_timing}")
                        else:
                            print(f"   {section_num}.{idx} {assessment}")
                elif isinstance(assessments, dict):
                    for idx, (key, value) in enumerate(assessments.items(), 1):
                        print(f"   {section_num}.{idx} {key}: {value}")
                section_num += 1
        
        # Time allocation
        if 'time_allocation' in plan_content or 'schedule' in plan_content:
            timing = plan_content.get('time_allocation') or plan_content.get('schedule', {})
            if timing:
                print(f"\n{section_num}. TIME ALLOCATION")
                if isinstance(timing, dict):
                    for idx, (section, time) in enumerate(timing.items(), 1):
                        print(f"   {section_num}.{idx} {section}: {time}")
                elif isinstance(timing, list):
                    for idx, item in enumerate(timing, 1):
                        print(f"   {section_num}.{idx} {item}")
    
    # If it's unstructured text, just display it
    elif isinstance(plan_content, str):
        print("\nPLAN CONTENT:")
        print(plan_content)
    
    # Content analysis summary
    content_analysis = lesson_plan.get('content_analysis', {})
    if content_analysis and content_analysis.get('total_sections'):
        print("\n" + "-"*80)
        print("\nCONTENT ANALYSIS:")
        print(f"   Total sections available: {content_analysis.get('total_sections')}")
        print(f"   Topics: {', '.join(content_analysis.get('topics', [])[:5])}")
        print(f"   Difficulty levels: {', '.join(content_analysis.get('difficulty_levels', []))}")
    
    print("\n" + "="*80 + "\n")

print("Hierarchical lesson plan display function created!")
print("Usage: display_lesson_plan_hierarchical(lesson_plan)")

In [ ]:
# Test 1: Process Rich Dad Poor Dad PDF
print("="*80)
print("TEST 1: PDF PROCESSING")
print("="*80)

pdf_path = r"c:\Users\dhanu\OneDrive\Desktop\last MAS\Rich Dad Poor Dad.pdf"

print(f"\nProcessing: Rich Dad Poor Dad.pdf")
print("Please wait...\n")

try:
    text_chunks, images = pdf_processor.process_pdf(pdf_path)
    pdf_processor.ingest_to_qdrant(text_chunks, images)
    
    print("\nPDF processing complete!")
    print(f"Stored {len(text_chunks)} text chunks in Qdrant")
    print(f"Stored {len(images)} images in Qdrant")
    print("\nSample content from first chunk:")
    if text_chunks:
        print(text_chunks[0]['payload']['text'][:200] + "...")
    
except Exception as e:
    print(f"\nError: {str(e)}")
    print("\nMake sure you've run all previous cells (3-18) to initialize the classes.")
    import traceback
    traceback.print_exc()

In [ ]:
# Test 2: Generate Lesson Plan
print("\n" + "="*80)
print("TEST 2: LESSON PLAN GENERATION")
print("="*80)

try:
    lesson_plan = planner.generate_lesson_plan(
        learning_objectives="Master financial literacy concepts from Rich Dad Poor Dad",
        target_duration=120,
        difficulty_preference="progressive"
    )
    
    if lesson_plan.get('status') == 'error':
        print(f"Error: {lesson_plan.get('message')}")
    else:
        print("\nLesson plan generated successfully!")
        print(f"Version: {lesson_plan.get('version')}")
        
except Exception as e:
    print(f"\nError generating lesson plan: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test 3: Display Lesson Plan in Hierarchical Format
print("\n" + "="*80)
print("TEST 3: HIERARCHICAL LESSON PLAN DISPLAY")
print("="*80)

try:
    if 'lesson_plan' in dir() and lesson_plan:
        display_lesson_plan_hierarchical(lesson_plan)
    else:
        print("\nNo lesson plan available. Run Test 2 first.")
        
except Exception as e:
    print(f"\nError displaying lesson plan: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# Debug: Check the actual lesson plan structure
if 'lesson_plan' in dir() and lesson_plan:
    print("Lesson plan keys:", lesson_plan.keys())
    print("\nPlan content type:", type(lesson_plan.get('plan')))
    print("\nPlan content preview:")
    plan_content = lesson_plan.get('plan', {})
    if isinstance(plan_content, dict):
        print("Keys:", plan_content.keys())
        import json
        print(json.dumps(plan_content, indent=2))
    else:
        print(str(plan_content))

In [ ]:
# Quick view: Show lesson plan structure
if 'lesson_plan' in dir() and lesson_plan:
    plan = lesson_plan.get('plan', {})
    
    # Check for lessons
    if 'lessons' in plan:
        lessons = plan['lessons']
        print(f"Found {len(lessons)} lessons in the plan:")
        print("\nLesson Structure:")
        for lesson in lessons[:5]:  # Show first 5
            lesson_num = lesson.get('lessonNumber', '?')
            title = lesson.get('title', 'No title')
            subsections = lesson.get('subsections', [])
            has_quiz = lesson.get('quizAfter', False)
            
            print(f"\n{lesson_num}. {title}")
            print(f"   Subsections: {len(subsections)}")
            if subsections:
                for sub in subsections[:3]:  # Show first 3
                    print(f"   - {sub.get('number', '')}: {sub.get('title', '')}")
            if has_quiz:
                print(f"   [QUIZ AFTER THIS LESSON]")
        
        if len(lessons) > 5:
            print(f"\n... and {len(lessons) - 5} more lessons")
    else:
        print("No 'lessons' key found in plan")
        print("Plan keys:", plan.keys())

In [ ]:
# Example: Complete Teaching Flow with Lesson Plan
print("="*80)
print("EXAMPLE: TEACHING FLOW FOLLOWING LESSON PLAN")
print("="*80)

# Get first lesson from plan
if 'lesson_plan' in dir() and lesson_plan:
    plan = lesson_plan.get('plan', {})
    lessons = plan.get('lessons', [])
    
    if lessons:
        first_lesson = lessons[0]
        print(f"\nLesson {first_lesson.get('lessonNumber')}: {first_lesson.get('title')}")
        print(f"Duration: {first_lesson.get('duration')} minutes")
        print(f"\nSubsections to teach:")
        
        for subsection in first_lesson.get('subsections', []):
            sub_num = subsection.get('number')
            sub_title = subsection.get('title')
            seq_start = subsection.get('sequenceStart', 0)
            seq_end = subsection.get('sequenceEnd', 10)
            
            print(f"\n  {sub_num} {sub_title}")
            print(f"      Content sections: {seq_start}-{seq_end}")
            
            # Show how to retrieve this content
            print(f"      To teach: session.teach_topic('{sub_title}', sequence_id={seq_start})")
        
        print(f"\n  After completing all subsections:")
        if first_lesson.get('quizAfter'):
            print(f"      Run: quiz = session.checkpoint_quiz('{first_lesson.get('title')}')")
            print(f"      Then: session.submit_quiz_answers(quiz['quiz_id'], answers)")
        
        print("\n" + "="*80)
        print("This creates a structured teaching session following the actual PDF content!")
        print("="*80)


In [ ]:
# Test 4: Test Student Session
print("\n" + "="*80)
print("TEST 4: ADAPTIVE TUTORING SESSION")
print("="*80)

try:
    # Initialize a test student
    test_student = "student_001"
    print(f"\nInitializing session for: {test_student}")
    
    # Create session
    session = AdaptiveTutoringSessionV2(test_student, lesson_plan if 'lesson_plan' in dir() else None)
    
    print("\nSession initialized successfully!")
    print("Student profile created")
    print("\nYou can now interact with the session using:")
    print("- session.teach_topic('topic_name')")
    print("- session.process_student_input('student message')")
    print("- session.checkpoint_quiz('topic_name')")
    
except Exception as e:
    print(f"\nError initializing session: {str(e)}")
    import traceback
    traceback.print_exc()